In [ ]:
# MNIST 로 CNN 연습
import tensorflow as tf
from tensorflow.keras import datasets, models, layers, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
print(train_images.shape)

# CNN : 3차원을 4차원으로 구조 변경(채널 추가)
train_images = train_images.reshape((60000, 28, 28, 1))
print(train_images.shape, train_images.ndim)
train_images = train_images / 255.0
# print(train_images[0])
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images / 255.0
print(train_labels[:3])

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000, 28, 28, 1) 4
[5 0 4]


In [ ]:
# 데이터 섞기
import numpy as np
x = np.random.sample((5, 2))
print(x)
dset = tf.data.Dataset.from_tensor_slices(x)
print(dset)
# 버퍼의 크기는 여유있게 크게 주는게 좋다. 데이터 갯수보다 많이 하는게 좋음. 
dset = tf.data.Dataset.from_tensor_slices(x).shuffle(100).batch(2)    # batch(2) 묶음 수
print(dset)
for a in dset:
  print(a)

[[0.55934871 0.76513513]
 [0.34117896 0.52255886]
 [0.98673724 0.6138614 ]
 [0.50428399 0.54350374]
 [0.11727763 0.24743795]]
<TensorSliceDataset shapes: (2,), types: tf.float64>
<BatchDataset shapes: (None, 2), types: tf.float64>
tf.Tensor(
[[0.11727763 0.24743795]
 [0.34117896 0.52255886]], shape=(2, 2), dtype=float64)
tf.Tensor(
[[0.50428399 0.54350374]
 [0.98673724 0.6138614 ]], shape=(2, 2), dtype=float64)
tf.Tensor([[0.55934871 0.76513513]], shape=(1, 2), dtype=float64)


In [ ]:
# MNIST의 train data 섞기
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(60000).batch(28)
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(28)
print(train_ds)
print(test_ds)

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float64, tf.uint8)>
<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float64, tf.uint8)>


In [ ]:
# 모델 생성방법 - subclassing API
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(filters=32, kernel_size=[3, 3], padding='valid', activation='relu')
    self.pool1 = MaxPool2D((2, 2))

    self.conv2 = Conv2D(filters=32, kernel_size=[3, 3], padding='valid', activation='relu')
    self.pool2 = MaxPool2D((2, 2))

    self.flatten = Flatten(dtype='float32')

    self.d1 = Dense(64, activation='relu')
    self.drop1 = Dropout(rate=0.3)
    self.d2 = Dense(10, activation='softmax')
  
  # call 이 init의 내용을 사용함
  def call(self, inputs):
    net = self.conv1(inputs)
    net = self.pool1(net)
    net = self.conv2(net)
    net = self.pool2(net)
    net = self.flatten(net)
    net = self.d1(net)
    net = self.drop1(net)
    net = self.d2(net)
    return net

model = MyModel()
tmp_inputs = tf.keras.Input(shape = (28, 28, 1))
model(tmp_inputs)
print(model.summary())

Model: "my_model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            multiple                  320       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 multiple                  0         
_________________________________________________________________
conv2d_5 (Conv2D)            multiple                  9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 multiple                  0         
_________________________________________________________________
flatten_2 (Flatten)          multiple                  0         
_________________________________________________________________
dense_4 (Dense)              multiple                  51264     
_________________________________________________________________
dropout_2 (Dropout)          multiple                  0

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# 일반적인 모델 학습 방법1
model.compile(optimizer=optimizer, loss=loss_object, metrics=['acc'])
model.fit(train_images, train_labels, batch_size=128, epochs=5, verbose=2, max_queue_size=10, workers=1, use_multiprocessing=True)
score = model.evaluate(test_images, test_labels)
print('test loss : ', score[0])
print('test acc : ', score[1])

import numpy as np
print('예측 값 : ', np.argmax(model.predict(test_images[:2]), 1))
print('실제 값 : ', test_labels[:2])

Epoch 1/5
469/469 - 34s - loss: 0.3596 - acc: 0.8901
Epoch 2/5
469/469 - 1s - loss: 0.1083 - acc: 0.9672
Epoch 3/5
469/469 - 1s - loss: 0.0804 - acc: 0.9754
Epoch 4/5
469/469 - 1s - loss: 0.0671 - acc: 0.9797
Epoch 5/5
469/469 - 1s - loss: 0.0541 - acc: 0.9832
313/313 [==============================] - 1s 2ms/step - loss: 0.0362 - acc: 0.9886
test loss :  0.036177534610033035
test acc :  0.9886000156402588
예측 값 :  [7 2]
실제 값 :  [7 2]


In [ ]:
# 모델 학습 방법 2 : GradientTape 사용
train_loss = tf.keras.metrics.Mean()
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

test_loss = tf.keras.metrics.Mean()
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  train_loss(loss)      # 가중치 평균 계산
  train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

EPOCHS = 5
for epoch in range(EPOCHS):
  for train_images, train_labels in train_ds:
    train_step(train_images, train_labels)
  
  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)
  
  template = 'epochs: {}, train_loss:{}, train_acc:{}, test_loss:{}, test_acc:{}'
  print(template.format(epoch + 1, train_loss.result(), train_accuracy.result()*100, 
                        test_loss.result(), test_accuracy.result()*100))
  
print('예측 값 : ', np.argmax(model.predict(test_images[:2]), 1))
print('실제 값 : ', test_labels[:2].numpy())

epochs: 1, train_loss:0.010421371087431908, train_acc:99.63999938964844, test_loss:0.04149317368865013, test_acc:98.95999908447266
epochs: 2, train_loss:0.0099940225481987, train_acc:99.65499877929688, test_loss:0.03994537144899368, test_acc:99.01000213623047
epochs: 3, train_loss:0.009059558622539043, train_acc:99.69000244140625, test_loss:0.03963252529501915, test_acc:99.03666687011719
epochs: 4, train_loss:0.008747817948460579, train_acc:99.70249938964844, test_loss:0.03996562957763672, test_acc:99.05750274658203
epochs: 5, train_loss:0.008145938627421856, train_acc:99.72166442871094, test_loss:0.04254552349448204, test_acc:99.01200103759766
예측 값 :  [3 4]
실제 값 :  [3 4]
